In [1]:
from magnet.base import Magnet
from magnet.base import EmbeddedMagnet

cluster = EmbeddedMagnet()
cluster.start()
magnet = cluster.create_magnet()
await magnet.align()

config = {
    "host": "127.0.0.1",
    "credentials": None,
    "domain": None,
    "name": "my_stream",
    "category": "my_category",
    "kv_name": "my_kv",
    "session": "my_session",
    "os_name": "my_object_store",
    "index": {
        "milvus_uri": "127.0.0.1",
        "milvus_port": 19530,
        "milvus_user": "test",
        "milvus_password": "test",
        "dimension": 1024,
        "model": "BAAI/bge-large-en-v1.5",
        "name": "test",
        "options": {
            'metric_type': 'COSINE',
            'index_type':'HNSW',
            'params': {
                "efConstruction": 40
                , "M": 48
            }
        }
    }
}

magnet = Magnet(config)
await magnet.align()

🌊 SUCCESS: containerization engine connected
☕️ WAIT: <Container: 27f20e0d3448>
☕️ WAIT: magnet-embedded-nats container progressing with id 27f20e0d34480c799e2dd0762b1e383bc0633365e5c01a5cc8ae86f076ad3883
ℹ️ INFO: nats logs
 📊 ⣽: [1] 2024/03/13 15:13:00.411527 
 🏁 ⣾:  Starting nats-server
[1] 2024/03/13 15:13:00.411607 
 💙 ⣯:    Version:  2.10.12
[1] 2024/03/13 15:13:00.411608 
 💚 ⡿:    Git:      [121169ea]
[1] 2024/03/13 15:13:00.411609 
 🧮 ⢿:    Name:     NDKBXN7NLCYKB5AH6Q6IZZUXYKXBP6FFCNZWZWTDDCMZATZPITIEL32N
[1] 2024/03/13 15:13:00.411613 
 🐈 ⣯:    Node:     fhL5pDzE
[1] 2024/03/13 15:13:00.411614 
 🧠 ⣷:    ID:       NDKBXN7NLCYKB5AH6Q6IZZUXYKXBP6FFCNZWZWTDDCMZATZPITIEL32N
[1] 2024/03/13 15:13:00.411796 
 🤩 ⣽:  Starting JetStream
[1] 2024/03/13 15:13:00.415358 
 💜 ⣟:      _ ___ _____ ___ _____ ___ ___   _   __  __
[1] 2024/03/13 15:13:00.415376 
 🧡 ⣷:   _ | | __|_   _/ __|_   _| _ \ __| /_\ |  \/  |
[1] 2024/03/13 15:13:00.415377 
 🧠 ⢿:  | || | _|  | | \__ \ | | |   / _| / _ \| |\

In [2]:
from magnet.ic.field import Charge

field = Charge(magnet)
await field.on()

🌊 SUCCESS: created [my_category] on
🛰️ stream: my_stream
🌊 SUCCESS: ready [my_category] on
🛰️ stream: my_stream


In [3]:
# you will get errors without abiding by schemas
await field.pulse("Hello, World!")
await field.pulse({"Hello": "World!"})

☠️ FATAL: invalid object, more info:
asdict() should be called on dataclass instances in [Payload, GeneratedPayload, EmbeddingPayload, JobParams]
☠️ FATAL: invalid object, more info:
asdict() should be called on dataclass instances in [Payload, GeneratedPayload, EmbeddingPayload, JobParams]


In [4]:
# simple data pipelines with predictable outcomes
from magnet.utils.data_classes import Payload
from time import sleep
no_log = await field.pulse(Payload("Hello", "World!"))
sleep(1)
msg = await field.pulse(Payload("Hello", "World again!"), True)
sleep(1)
duplicate = await field.pulse(Payload("Hello", "World again!"), True) # will show as duplicate
print(no_log.ts, msg.ts, duplicate.ts)
(no_log, msg, duplicate)

🌊 SUCCESS: pulsed to my_category on my_stream
🌊 SUCCESS: pulsed to my_category on my_stream
2024-03-13 15:13:07.805490+00:00 2024-03-13 15:13:08.813173+00:00 2024-03-13 15:13:09.816282+00:00


(PubAck(stream='my_stream', seq=1, domain=None, duplicate=None),
 PubAck(stream='my_stream', seq=2, domain=None, duplicate=None),
 PubAck(stream='my_stream', seq=2, domain=None, duplicate=True))

In [5]:
from magnet.ize.memory import Memory
mem = Memory(magnet)
await mem.on(create=True)


🚨 WARN: CUDA is not available on this machine.
ℹ️ INFO: loading into cpu
☕️ WAIT: connecting to 127.0.0.1
🌊 SUCCESS: connected successfully to 127.0.0.1
🌊 SUCCESS: test created
☕️ WAIT: loading test into memory, may take time


In [6]:
await mem.on()

from magnet.ic.field import Resonator

reso = Resonator(magnet)

async def handle_payload(payload, msg):
    await mem.index(payload, msg, v=True)

await reso.on()
await reso.listen(cb=handle_payload)

🚨 WARN: CUDA is not available on this machine.
ℹ️ INFO: loading into cpu
☕️ WAIT: connecting to 127.0.0.1
🌊 SUCCESS: connected successfully to 127.0.0.1
☕️ WAIT: loading test into memory, may take time
☕️ WAIT: connecting to 127.0.0.1
ℹ️ INFO: joined worker queue: my_session as unifi
ℹ️ INFO: consuming delta from [my_category] on
🛰️ stream: my_stream
🧲 session: "my_session"
ℹ️ INFO: encoding payload
Payload(text='Hello', document='World!')
ℹ️ INFO: indexing payload
🌊 SUCCESS: embedding indexed
Payload(text='Hello', document='World!')
ℹ️ INFO: encoding payload
Payload(text='Hello', document='World again!')
ℹ️ INFO: indexing payload
🌊 SUCCESS: embedding indexed
Payload(text='Hello', document='World again!')
🚨 WARN: no more data


In [7]:
mem.db.list_indices()

await mem.disconnect()
cluster.stop()
cluster.cleanup()

🚨 WARN: disconnected from 127.0.0.1
☕️ WAIT: stopping magnet-embedded-nats
🌊 SUCCESS: magnet-embedded-nats stopped
ℹ️ INFO: removing magnet-embedded-nats
🌊 SUCCESS: embedded nats removed
🌊 SUCCESS: embedded milvus server stopped
🚨 WARN: container engine pruned
🌊 SUCCESS: embedded cluster cleaned up
